In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

In [3]:
conn=wrds.Connection()

Enter your WRDS username [mmw]:dachxiu
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [35]:
fyear = '2016'

comp = conn.raw_sql(f"""
                        select gvkey, datadate, at, pstkl, txditc,
                        pstkrv, seq, pstk, ni
                        from comp.funda
                        where indfmt='INDL' 
                        and datafmt='STD'
                        and popsrc='D'
                        and consol='C'
                        and fyear>='{2016}'
                        """)

In [25]:
comp.head(5)

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk
0,001004,2020-05-31,2079.000,0.0,0.000,0.0,902.600,0.0
1,001119,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,001045,2019-12-31,59995.000,0.0,29.000,0.0,-118.000,0.0
3,001352,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN
4,001050,2019-12-31,408.637,0.0,5.943,0.0,193.017,0.0


In [24]:
comp[comp['gvkey']=='335466']

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk
10516,335466,2019-12-31,201.909,0.0,0.0,0.0,52.128,0.0


In [29]:
# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])

comp['txditc']=comp['txditc'].fillna(0)

# create book equity
comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

In [31]:
comp['datadate']=pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year']=comp['datadate'].dt.year

# number of years in Compustat
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

comp=comp[['gvkey','datadate','year','be','count']]

In [32]:
comp.head(10)

,gvkey,datadate,year,be,count
0,001004,2020-05-31,2020,902.600,0
2,001045,2019-12-31,2019,NaN,0
4,001050,2019-12-31,2019,198.960,0
5,001062,2019-11-30,2019,285.879,0
7,001075,2019-12-31,2019,7619.455,0
8,001076,2019-12-31,2019,2047.654,0
9,001078,2019-12-31,2019,32634.000,0
10,001084,2019-12-31,2019,NaN,0
11,001096,2019-12-31,2019,4282.692,0
12,001104,2019-12-31,2019,55.754,0
